Welcome to day 1 of the 5-Day Data Challenge! Today, we're going to be looking at how to deal with missing values. To get started, click the blue "Fork Notebook" button in the upper, right hand corner. This will create a private copy of this notebook that you can edit and play with. Once you're finished with the exercises, you can choose to make your notebook public to share with others. :)

> **Your turn!** As we work through this notebook, you'll see some notebook cells (a block of either code or text) that has "Your Turn!" written in it. These are exercises for you to do to help cement your understanding of the concepts we're talking about. Once you've written the code to answer a specific question, you can run the code by clicking inside the cell (box with code in it) with the code you want to run and then hit CTRL + ENTER (CMD + ENTER on a Mac). You can also click in a cell and then click on the right "play" arrow to the left of the code. If you want to run all the code in your notebook, you can use the double, "fast forward" arrows at the bottom of the notebook editor.

Here's what we're going to do today:

* [Take a first look at the data](#Take-a-first-look-at-the-data)
* [See how many missing data points we have](#See-how-many-missing-data-points-we-have)
* [Figure out why the data is missing](#Figure-out-why-the-data-is-missing)
* [Drop missing values](#Drop-missing-values)
* [Filling in missing values](#Filling-in-missing-values)

Let's get started!

# Take a first look at the data
________

The first thing we'll need to do is load in the libraries and datasets we'll be using. For today, I'll be using a dataset of events that occured in American Football games for demonstration, and you'll be using a dataset of building permits issued in San Francisco.

> **Important!** Make sure you run this cell yourself or the rest of your code won't work!

**NOTE FOR MY KERNEL**  I have removed the nfl_dataset that Rachael uses for her demonstration for clarity as we are supposed to run a similar analysis on the aforementioned San Francisco building permits dataset.  I did however keep all of her code and  commentary and some references to the American Football dataset with my comments on the San Francisco building permits dataset in **bold**.   Be sure to check out Rachaels original kernel if you want to see the NFL dataset examples.

In [1]:
# modules we'll use
import pandas as pd
import numpy as np

# read in all our data
sf_permits = pd.read_csv("../input/building-permit-applications-data/Building_Permits.csv", low_memory=False)

# set seed for reproducibility
np.random.seed(0) 

The first thing I do when I get a new dataset is take a look at some of it. This lets me see that it all read in correctly and get an idea of what's going on with the data. In this case, I'm looking to see if I see any missing values, which will be reprsented with `NaN` or `None`.

In [2]:
# look at a few rows of the sf_permits file. There is a handful of missing data already!
sf_permits.sample(5)

**Personnally I like to use the methods 'info()' and 'describe()' when perusing a newly imported dataset.  The info() method gives you an idea of how many non-null (and conversely how many null) entries exist and also (if the dataset isn't too large) gives you a look at the feature names to see what kind of information makes up the dataset.  The describe() method gives a more statistical breakdown of the dataset which is useful for such numerical features such as 'Estimated Cost'.**

In [4]:
sf_permits.info()
#Using describe gives you more statistical information
sf_permits.describe()

# See how many missing data points we have
___

Ok, now we know that we do have some missing values. Let's see how many we have in each column. 
>**Since there are a total of 198900 rows in the sf_permits dataset counting the null entries you should see the inverse of what you found with the info() method; i.e. permits have 198900 non-null entries so the missing values count is equal to 0.  Counting the null entries does save you from having to do subtraction in your head though!**

In [5]:
# get the number of missing data points per column
missing_values_count = sf_permits.isnull().sum()

# look at the # of missing points
missing_values_count

That seems like a lot! It might be helpful to see what percentage of the values in our dataset were missing to give us a better sense of the scale of this problem:
>**This is true for the NFL dataset and for some features of the sf_permits dataset and there is enough missing to warrant a look at how much it adds up to.**

In [6]:
# how many total missing values do we have?
total_cells = np.product(sf_permits.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
(total_missing/total_cells) * 100

Wow, almost a quarter of the cells in this dataset are empty! In the next step, we're going to take a closer look at some of the columns with missing values and try to figure out what might be going on with them.
>**You can see this is slightly over a quarter for the sf_permits dataset.  This is also the point in the process where one needs to examine the features to determine if trying to figure out why data is missing is worth the effort; the question to be asked is, "Can I do without this feature and what impact would not including it have on my analysis?"  This is what Rachel discusses below as 'data intuition'.**

# Figure out why the data is missing
____
 
This is the point at which we get into the part of data science that I like to call "data intution", by which I mean "really looking at your data and trying to figure out why it is the way it is and how that will affect your analysis". It can be a frustrating part of data science, especially if you're newer to the field and don't have a lot of experience. For dealing with missing values, you'll need to use your intution to figure out why the value is missing. One of the most important question you can ask yourself to help figure this out is this:

> Is this value missing becuase it wasn't recorded or becuase it dosen't exist?

If a value is missing becuase it doens't exist (like the height of the oldest child of someone who doesn't have any children) then it doesn't make sense to try and guess what it might be. These values you probalby do want to keep as NaN. On the other hand, if a value is missing becuase it wasn't recorded, then you can try to guess what it might have been based on the other values in that column and row. (This is called "imputation" and we'll learn how to do it next! :)

Let's work through an example. Looking at the number of missing values in the nfl_data dataframe, I notice that the column `TimesSec` has a lot of missing values in it: 

>**The same is true for the sf_permits dataset - which features matter and which don't?  Read through Rachael's discussion and tips below and then we will continue with the sf_permits analysis. **

By looking at [the documentation](https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016), I can see that this column has information on the number of seconds left in the game when the play was made. This means that these values are probably missing because they were not recorded, rather than because they don't exist. So, it would make sense for us to try and guess what they should be rather than just leaving them as NA's.

On the other hand, there are other fields, like `PenalizedTeam` that also have lot of missing fields. In this case, though, the field is missing because if there was no penalty then it doesn't make sense to say *which* team was penalized. For this column, it would make more sense to either leave it empty or to add a third value like "neither" and use that to replace the NA's.

> **Tip:** This is a great place to read over the dataset documentation if you haven't already! If you're working with a dataset that you've gotten from another person, you can also try reaching out to them to get more information.

If you're doing very careful data analysis, this is the point at which you'd look at each column individually to figure out the best strategy for filling those missing values. For the rest of this notebook, we'll cover some "quick and dirty" techniques that can help you with missing values but will probably also end up removing some useful information or adding some noise to your data.

## Your turn!

* Look at the columns `Street Number Suffix` and `Zipcode` from the `sf_permits` datasets. Both of these contain missing values. Which, if either, of these are missing because they don't exist? Which, if either, are missing because they weren't recorded?

In [7]:
sf_permits[['Street Number Suffix', 'Zipcode']].sample(10)

**Now I'm no street address expert guy but Wikipedia is (https://en.wikipedia.org/wiki/Street_suffix).  Apparently there are 148 street number suffix's recognized by the U.S. Postal Service that provide additional information about the address (i.e. the suffix Ave. stands for avenue) but it doesn't seem very standardized so I think that the missing information was simply not recorded...especially when you consider that only a small fraction is actually filled in and it seems to be repeated in the column 'Street Suffix'.  However, I think the zipcode is definitely pertinent information for a permit so something needs to be done about the missing values in this feature.**

# Drop missing values
___

If you're in a hurry or don't have a reason to figure out why your values are missing, one option you have is to just remove any rows or columns that contain missing values. (Note: I don't generally recommend this approch for important projects! It's usually worth it to take the time to go through your data and really look at all the columns with missing values one-by-one to really get to know your dataset.)  

If you're sure you want to drop rows with missing values, pandas does have a handy function, `dropna()` to help you do this. Let's try it out on our NFL dataset!
>**...or let's try it on the sf_permits dataset!**

In [8]:
# remove all the rows that contain a missing value
sf_permits.dropna()

Oh dear, it looks like that's removed all our data! 😱 This is because every row in our dataset had at least one missing value. We might have better luck removing all the *columns* that have at least one missing value instead.
>**Same thing happens with the sf_permits dataset as you can see above.  Let's work on the columns instead of rows of the sf_permits dataset.  Remember, setting axis=1 defines columns instead of the default rows (which axis=0 but you don't have to type it in since it is the default).**

In [9]:
# remove all columns with at least one missing value
columns_with_na_dropped = sf_permits.dropna(axis=1)
columns_with_na_dropped.head()

In [10]:
# just how much data did we lose?
print("Columns in original dataset: %d \n" % sf_permits.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

We've lost quite a bit of data, but at this point we have successfully removed all the `NaN`'s from our data. 

# Filling in missing values automatically
_____

Another option is to try and fill in the missing values. For this next bit, I'm getting a small sub-section of the NFL data so that it will print well.
>**We will do the same thing with the sf_permits data instead and start with the column of 'Supervisor District' to the column 'Location' with four columns in total.  The tail of the dataset is where you will find NaN's for these columns.**

In [39]:
# get a small subset of the sf_permits dataset
subset_sf_permits = sf_permits.loc[:, 'Supervisor District':'Location'].tail(20)
subset_sf_permits

We can use the Panda's fillna() function to fill in missing values in a dataframe for us. One option we have is to specify what we want the `NaN` values to be replaced with. Here, I'm saying that I would like to replace all the `NaN` values with 0.

In [40]:
# replace all NA's with 0
subset_sf_permits.fillna("0")

I could also be a bit more savvy and replace missing values with whatever value comes directly after it in the same column. (This makes a lot of sense for datasets where the observations have some sort of logical order to them.)
>**For this particular subset forward fill ('ffill') works better than backfill ('bfill').**

In [45]:
# replace all NA's the value that comes directly after it in the same column, 
# then replace all the reamining na's with 0
subset_sf_permits.fillna(method = 'ffill', axis=0).fillna("0")

Filling in missing values is also known as "imputation", and you can find more exercises on it [in this lesson, also linked under the "More practice!" section](https://www.kaggle.com/dansbecker/handling-missing-values). 

And that's it for today! If you have any questions, be sure to post them in the comments below or [on the forums](https://www.kaggle.com/questions-and-answers). 

Remember that your notebook is private by default, and in order to share it with other people or ask for help with it, you'll need to make it public. First, you'll need to save a version of your notebook that shows your current work by hitting the "Commit & Run" button. (Your work is saved automatically, but versioning your work lets you go back and look at what it was like at the point you saved it. It also let's you share a nice compiled notebook instead of just the raw code.) Then, once your notebook is finished running, you can go to the Settings tab in the panel to the left (you may have to expand it by hitting the [<] button next to the "Commit & Run" button) and setting the "Visibility" dropdown to "Public".

# More practice!
___

If you're looking for more practice handling missing values, check out these extra-credit\* exercises:

* [Handling Missing Values](https://www.kaggle.com/dansbecker/handling-missing-values): In this notebook Dan shows you several approaches to imputing missing data using scikit-learn's imputer. 
* Look back at the `Zipcode` column in the `sf_permits` dataset, which has some missing values. How would you go about figuring out what the actual zipcode of each address should be? (You might try using another dataset, like the [OpenAddresses dataset](https://www.kaggle.com/openaddresses/openaddresses-us-west).) 

\* no actual credit is given for completing the challenge, you just learn how to clean data real good :P